<a href="https://colab.research.google.com/github/mianyumifen-bot/codePublic/blob/main/%E7%AC%AC7%E5%A4%A9sential700%E7%B1%B3%E7%BC%93%E5%86%B2%E5%8C%BA%E4%B8%8B%E8%BD%BDNDWI(2025_11_05).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -*- coding: utf-8 -*-
# ======================================================
#  第1单元格：安装必要的库
# ======================================================
# 这个命令会安装所有处理地理数据和与Google Earth Engine交互所需的库
# --quiet 参数可以减少安装过程中的输出信息
!pip install --quiet earthengine-api geemap rasterio rioxarray geopandas shapely pyproj

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 64.6 MB/s eta 0:00:00


In [3]:
# -*- coding: utf-8 -*-
# ======================================================
#  第2单元格：导入、授权与任务清理
# ======================================================

# 导入标准库
import os
import time
import json
import math
import datetime

# 导入地理空间处理和GEE库
import geopandas as gpd
import pandas as pd
import ee
import geemap
from shapely.geometry import mapping

# --- 授权并初始化GEE ---
try:
    # 填入你自己的GEE项目名称
    ee.Initialize(project="ee-mianyumifen")
except Exception as e:
    print("需要进行GEE授权，请根据提示操作...")
    ee.Authenticate()
    ee.Initialize(project="ee-mianyumifen")
print("Google Earth Engine 已成功初始化。")


# --- 清理正在运行或等待中的GEE任务 (可选但推荐) ---
print("\n正在获取并取消所有正在运行或等待中的GEE任务...")
try:
    tasks = ee.batch.Task.list()
    canceled_count = 0
    for task in tasks:
        if task.state == 'RUNNING' or task.state == 'READY':
            task.cancel()
            canceled_count += 1
    if canceled_count > 0:
        print(f"已成功发送 {canceled_count} 个任务的取消请求。")
    else:
        print("没有找到正在运行或等待中的任务。")
except Exception as e:
    print(f"获取任务列表失败: {e}")


# --- 挂载Google Drive ---
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
print("\n谷歌硬盘已成功挂载。")

需要进行GEE授权，请根据提示操作...
Google Earth Engine 已成功初始化。

正在获取并取消所有正在运行或等待中的GEE任务...
没有找到正在运行或等待中的任务。
Mounted at /content/drive

谷歌硬盘已成功挂载。


In [5]:
# -*- coding: utf-8 -*-
# ======================================================
#  第3单元格：配置参数
# ======================================================

# ========== 文件夹和文件路径 (请根据你的情况修改) ==========
DRIVE_INPUT_DIR = '/content/drive/MyDrive/allPoint'

# 定义一个“中转站”文件夹，所有从GEE导出的文件会先存放在这里
STAGING_EXPORT_FOLDER = 'S2_700_NDWI_Final_Exports'

# 定义最终整理好的文件的根目录 (后续处理脚本会用到)
FINAL_OUTPUT_ROOT_FOLDER = 'S2_700_NDWI_Yearly_Composites_Final'

# 日志文件路径
LOG_PATH = os.path.join(DRIVE_INPUT_DIR, 's2_yearly_ndwi_final_export_log.json')


# ========== 导出与处理控制参数 ==========
HLS_COLLECTION = 'COPERNICUS/S2_SR_HARMONIZED'
# 计算NDWI需要 B3(Green) 和 B8(NIR)
PREFERRED_BANDS = ['B3', 'B8']
EXPORT_SCALE = 10    # 导出影像的分辨率 (米)
BUFFER_RADIUS = 700  # 点周边的缓冲区半径 (米)```



# -*- coding: utf-8 -*-
# ======================================================
#  第4单元格：加载点位Shapefile数据
# ======================================================

# 构建shapefile的完整路径
roi_shp_path = os.path.join(DRIVE_INPUT_DIR, 'allPoint.shp')

# 检查文件是否存在
if not os.path.exists(roi_shp_path):
    raise FileNotFoundError(f"未在指定路径找到shapefile: {roi_shp_path}")

# 使用GeoPandas读取shapefile
rois_gdf = gpd.read_file(roi_shp_path)
print(f"成功从shapefile中加载了 {len(rois_gdf)} 个点。")

# 自动选择 'site' 或 'name' 字段作为地点名称
if 'site' in rois_gdf.columns:
    name_field = 'site'
elif 'name' in rois_gdf.columns:
    name_field = 'name'
else:
    name_field = 'site'
    rois_gdf[name_field] = [f"roi_{i}" for i in range(len(rois_gdf))]
    print("警告：未找到 'site' 或 'name' 字段，已自动生成默认ID。")

# 创建一个字典，格式为 {地点名称: 地理位置信息}
roi_dict = {str(row[name_field]): row.geometry for _, row in rois_gdf.iterrows()}
print("\nShapefile中的地点名称示例:", list(roi_dict.keys())[:10])

成功从shapefile中加载了 11 个点。

Shapefile中的地点名称示例: ['norriepoint', 'cmarshhighsaltmarsh', 'bnzrichfen', 'brackishimpoundment', 'gcesapelo', 'hillslough', 'mayberry', 'northinletsaltmarsh', 'richmondbrackishmarsh', 'siwetland']


In [7]:
# -*- coding: utf-8 -*-
# ======================================================
#  第5单元格：生成“站点-年份”处理列表
# ======================================================

# 如果列表为空，则自动处理所有从shapefile中读取的站点
target_sites = []
if not target_sites:
    target_sites = list(roi_dict.keys())

current_year = datetime.datetime.now().year
pairs = []  # 存储所有待处理的 (站点, 年份) 组合

print("\n正在查询GEE以确定每个站点的可用年份...")
for site in target_sites:
    if site not in roi_dict: continue

    ee_point = ee.Geometry(mapping(roi_dict[site]))
    ee_roi_for_check = ee_point.buffer(BUFFER_RADIUS)
    col = ee.ImageCollection(HLS_COLLECTION).filterBounds(ee_roi_for_check)

    if col.size().getInfo() == 0:
        print(f" -> {site} 周围未发现任何影像，已跳过。")
        continue

    try:
        first_year = ee.Image(col.first()).date().get('year').getInfo()
    except Exception as e:
        print(f" -> 无法确定 {site} 的首个年份，将默认从2015年开始。")
        first_year = 2015

    for y in range(first_year, current_year + 1):
        pairs.append((site, str(y)))

print(f"\n已生成总计 {len(pairs)} 个“站点-年份”组合待处理。")
print("队列中的前40个任务:", pairs[:40])


正在查询GEE以确定每个站点的可用年份...

已生成总计 118 个“站点-年份”组合待处理。
队列中的前40个任务: [('norriepoint', '2015'), ('norriepoint', '2016'), ('norriepoint', '2017'), ('norriepoint', '2018'), ('norriepoint', '2019'), ('norriepoint', '2020'), ('norriepoint', '2021'), ('norriepoint', '2022'), ('norriepoint', '2023'), ('norriepoint', '2024'), ('norriepoint', '2025'), ('cmarshhighsaltmarsh', '2015'), ('cmarshhighsaltmarsh', '2016'), ('cmarshhighsaltmarsh', '2017'), ('cmarshhighsaltmarsh', '2018'), ('cmarshhighsaltmarsh', '2019'), ('cmarshhighsaltmarsh', '2020'), ('cmarshhighsaltmarsh', '2021'), ('cmarshhighsaltmarsh', '2022'), ('cmarshhighsaltmarsh', '2023'), ('cmarshhighsaltmarsh', '2024'), ('cmarshhighsaltmarsh', '2025'), ('bnzrichfen', '2018'), ('bnzrichfen', '2019'), ('bnzrichfen', '2020'), ('bnzrichfen', '2021'), ('bnzrichfen', '2022'), ('bnzrichfen', '2023'), ('bnzrichfen', '2024'), ('bnzrichfen', '2025'), ('brackishimpoundment', '2015'), ('brackishimpoundment', '2016'), ('brackishimpoundment', '2017'), ('bracki

In [6]:
# -*- coding: utf-8 -*-
# ======================================================
#  第6单元格：辅助函数与云掩膜逻辑
# ======================================================

def shapely_to_ee(geom):
    """将Shapely库的geometry对象转换为GEE的ee.Geometry对象。"""
    return ee.Geometry(mapping(geom))

def make_mask_function(bandnames_list):
    """创建一个仅使用QA60波段进行云掩膜的函数。"""
    def mask_fn(img):
        qa = img.select('QA60')
        # 位10是不透明云, 位11是卷云
        opaque_cloud_bit = 1 << 10
        cirrus_bit = 1 << 11
        mask = qa.bitwiseAnd(opaque_cloud_bit).eq(0).And(qa.bitwiseAnd(cirrus_bit).eq(0))
        return img.updateMask(mask)
    return mask_fn

print("辅助函数和掩膜函数已定义。")

辅助函数和掩膜函数已定义。


In [8]:
# -*- coding: utf-8 -*-
# ======================================================
#  第7单元格：核心处理与导出函数 (最终命名版)
# ======================================================

def process_and_export_yearly_ndwi_stack_final(site_name, year, point_geom, staging_folder, scale=EXPORT_SCALE, radius=BUFFER_RADIUS):
    """
    处理“站点-年份”，将该年度所有晴空影像的NDWI计算结果堆叠成一个多波段影像。
    文件名格式: site_name_year.tif
    波段命名格式: 日期_NDWI_瓦片号
    """
    ee_point = shapely_to_ee(point_geom)
    ee_roi = ee_point.buffer(radius)

    start_date = f'{int(year)}-01-01'; end_date = f'{int(year)}-12-31'
    col = ee.ImageCollection(HLS_COLLECTION).filterDate(start_date, end_date).filterBounds(ee_roi)
    col_size = col.size().getInfo()
    print(f"[{site_name} {year}] 原始影像数量: {col_size}")
    if col_size == 0: return {'status': 'no_images_in_year', 'site': site_name, 'year': year}

    mask_fn = make_mask_function(None) # QA60掩膜函数

    # 定义函数，计算NDWI并按“日期_NDWI_瓦片号”格式重命名
    def calculate_and_rename_ndwi(img):
        img = ee.Image(img)
        img_masked = mask_fn(img)

        # 应用比例因子并计算NDWI (Green - NIR) / (Green + NIR)
        ndwi = img_masked.select(PREFERRED_BANDS).toFloat().multiply(0.0001) \
                         .normalizedDifference(['B3', 'B8'])

        # 提取元数据
        date_str = ee.Date(img.get('system:time_start')).format('YYYYMMdd')
        tile_id = ee.String(img.get('MGRS_TILE')) # 获取瓦片号

        # 按照“日期_NDWI_瓦片号”格式拼接新名称
        new_name = date_str.cat('_NDWI_').cat(tile_id)

        return ndwi.rename(new_name)

    prepared_col = col.map(calculate_and_rename_ndwi)
    prepared_size = prepared_col.size().getInfo()
    print(f"  经过掩膜后，可用于堆叠的NDWI影像数量: {prepared_size}")
    if prepared_size == 0: return {'status': 'no_clear_images', 'site': site_name, 'year': year}

    stacked_image = prepared_col.toBands().clip(ee_roi)

    # 使用简化的文件名 "sitename_year.tif"
    export_name = f"{site_name}_{year}"
    task = ee.batch.Export.image.toDrive(
        image=stacked_image,
        description=export_name,
        folder=staging_folder,
        fileNamePrefix=export_name,
        region=ee_roi,
        scale=scale,
        maxPixels=1e13
    )
    task.start()

    print(f"  -> 已提交任务: {export_name}.tif -> 中转文件夹: {staging_folder}")
    return {'status': 'export_started', 'site': site_name, 'year': year, 'task_id': task.id}

print("核心处理与导出函数已定义。")

核心处理与导出函数已定义。


In [9]:
# -*- coding: utf-8 -*-
# ======================================================
#  第8单元格：主循环 (开始执行)
# ======================================================

results = []
started_tasks_count = 0

for idx, (site, year) in enumerate(pairs, start=1):
    print(f"\n[{idx}/{len(pairs)}] 开始处理： {site} - {year}")
    if site not in roi_dict: continue
    try:
        res = process_and_export_yearly_ndwi_stack_final(
            site_name=site,
            year=year,
            point_geom=roi_dict[site],
            staging_folder=STAGING_EXPORT_FOLDER
        )
        results.append(res)
        if res.get('status') == 'export_started': started_tasks_count += 1
    except Exception as e:
        results.append({'site':site, 'year':year, 'status':'fatal_error', 'error_msg': str(e)})

# 保存日志
with open(LOG_PATH, 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print("\n========================================================")
print("所有 NDWI 堆叠影像导出任务已提交。")
print(f"本次运行共提交了 {started_tasks_count} 个导出任务。")
print("请访问GEE的Tasks页面监控导出进度。")
print("导出的文件名和波段名已是最终格式，无需任何修复！")
print("========================================================")


[1/118] 开始处理： norriepoint - 2015
[norriepoint 2015] 原始影像数量: 13
  经过掩膜后，可用于堆叠的NDWI影像数量: 13
  -> 已提交任务: norriepoint_2015.tif -> 中转文件夹: S2_700_NDWI_Final_Exports

[2/118] 开始处理： norriepoint - 2016
[norriepoint 2016] 原始影像数量: 67
  经过掩膜后，可用于堆叠的NDWI影像数量: 67
  -> 已提交任务: norriepoint_2016.tif -> 中转文件夹: S2_700_NDWI_Final_Exports

[3/118] 开始处理： norriepoint - 2017
[norriepoint 2017] 原始影像数量: 123
  经过掩膜后，可用于堆叠的NDWI影像数量: 123
  -> 已提交任务: norriepoint_2017.tif -> 中转文件夹: S2_700_NDWI_Final_Exports

[4/118] 开始处理： norriepoint - 2018
[norriepoint 2018] 原始影像数量: 142
  经过掩膜后，可用于堆叠的NDWI影像数量: 142
  -> 已提交任务: norriepoint_2018.tif -> 中转文件夹: S2_700_NDWI_Final_Exports

[5/118] 开始处理： norriepoint - 2019
[norriepoint 2019] 原始影像数量: 144
  经过掩膜后，可用于堆叠的NDWI影像数量: 144
  -> 已提交任务: norriepoint_2019.tif -> 中转文件夹: S2_700_NDWI_Final_Exports

[6/118] 开始处理： norriepoint - 2020
[norriepoint 2020] 原始影像数量: 145
  经过掩膜后，可用于堆叠的NDWI影像数量: 145
  -> 已提交任务: norriepoint_2020.tif -> 中转文件夹: S2_700_NDWI_Final_Exports

[7/118] 开始处理： norriepoint - 202

In [ ]:
# -*- coding: utf-8 -*-
# ======================================================
#  第9单元格：文件整理脚本
#  !!! 请在GEE任务全部完成后再运行此单元格 !!!
# ======================================================

import os
import shutil
from google.colab import drive

# 确保硬盘已挂载
drive.mount('/content/drive', force_remount=True)

# --- 核心配置 (这里的文件夹名必须与第3单元格中的完全一致！) ---
DRIVE_ROOT = '/content/drive/MyDrive'
STAGING_EXPORT_FOLDER = 'S2_700_NDWI_Final_Exports'      # 从这里读取
FINAL_OUTPUT_ROOT_FOLDER = 'S2_700_NDWI_Yearly_Composites_Final' # 移动到这里

staging_path = os.path.join(DRIVE_ROOT, STAGING_EXPORT_FOLDER)
final_path = os.path.join(DRIVE_ROOT, FINAL_OUTPUT_ROOT_FOLDER)
# ----------------------------------------------------

print(f"\n开始整理文件...")
print(f"源文件夹: {staging_path}")
print(f"目标文件夹: {final_path}")

# 确保最终的目标根目录存在
os.makedirs(final_path, exist_ok=True)
moved_count = 0

if not os.path.exists(staging_path):
    print(f"\n错误：找不到中转文件夹 '{staging_path}'。请检查名称是否正确，以及文件是否已从GEE导出。")
else:
    for filename in os.listdir(staging_path):
        if filename.endswith('.tif'):
            try:
                # 从 "sitename_year.tif" 中提取 "sitename"
                site_name = filename.split('_')[0]

                # 创建站点子文件夹，例如 /.../S2_700_NDWI_Yearly_Composites_Final/northinletsaltmarsh/
                site_folder = os.path.join(final_path, site_name)
                os.makedirs(site_folder, exist_ok=True)

                # 构建路径并移动
                source_file = os.path.join(staging_path, filename)
                destination_file = os.path.join(site_folder, filename)
                shutil.move(source_file, destination_file)
                print(f"  已移动: {filename} -> {site_folder}/")
                moved_count += 1
            except Exception as e:
                print(f"  处理文件 {filename} 时出错: {e}")

print("\n========================================================")
print(f"整理完成！共移动了 {moved_count} 个文件。")
if moved_count > 0:
    print(f"提示：中转文件夹 '{STAGING_EXPORT_FOLDER}' 中的文件已被移空，您可以根据需要手动删除该文件夹。")
print("========================================================")